# How to implement a YOLO (v3) object detector from scratch in PyTorch
[tutorial link](https://blog.paperspace.com/how-to-implement-a-yolo-object-detector-in-pytorch/)

YOLO stands for You Only Look Once. It's an object detector that uses features learned by a deep convolutional neural network to detect an object.

YOLO makes use of only convolutional layers, making it a fully convolutional network (FCN). It has 75 convolutional layers, with skip connections and upsampling layers. No form of pooling is used, and a convolutional layer with stride 2 is used to downsample the feature maps. This helps in preventing loss of low-level features often attributed to pooling.

Being a (fully convolutional network) FCN, YOLO is invariant to the size of the input image. However, in practice, we might want to stick to a constant input size due to various problems that only show their heads when we are implementing the algorithm.

### why you want all the image to be the same input size:
A big one amongst these problems is that if we want to process our images in batches (images in batches can be processed in parallel by the GPU, leading to speed boosts), we need to have all images of fixed height and width.

### Interpreting the output

Typically, (as is the case for all object detectors) the features learned by the convolutional layers are passed onto a classifier/regressor which makes the detection prediction (coordinates of the bounding boxes, the class label.. etc).

In YOLO, the prediction is done by using a convolutional layer which uses 1 x 1 convolutions.

Now, the first thing to notice is our output is a feature map (Neuron/cell). Since we have used 1 x 1 convolutions, the size of the prediction map is exactly the size of the feature map before it.

You expect each cell of the feature map to predict an object through one of it's bounding boxes if the center of the object falls in the receptive field of that cell.

Important concepts:

* Upsampling (torch.nn.Upsample): 

    * “Upsampling” is the process of inserting zero-valued samples between original samples to increase the sampling rate. (This is called “zero-stuffing”.) Upsampling adds to the original signal undesired spectral images which are centered on multiples of the original sampling rate.
    * [Excellent stackOverflow answer](https://datascience.stackexchange.com/questions/6107/what-are-deconvolutional-layers)

* Residual block [(resNet)](https://shuzhanfan.github.io/2018/11/ResNet/):

    * provide an answer to the vanishing gradient problem of deep neural network by adding the output of the previous layer (residual) to the output of the current layer and you do so for multiple layers.
    ![img](images/resnet1.jpg)
    * [Residual Block](https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278), PyTorch does not have the ‘auto’ padding in Conv2d so you have to code it yourself
    * paper in the folder

* vanishing/exploding Gradient:
    
    * if w are bigger than 1 with very deep network the backpropagation gradient may explode with very deep neural networks and vice versa for smaller weight.
    * partial solution is [better weight initialization](https://www.youtube.com/watch?v=s2coXdufOzE)
    * [Andrew Ng](https://www.youtube.com/watch?v=qhXZsFVxGKo)
    * the slopes/derivates becomes exponentionaly small or large 
* bounding Box prediction with PyTorch:
    * [code tutorial](https://towardsdatascience.com/bounding-box-prediction-from-scratch-using-pytorch-a8525da51ddc)

* [non-Maximum Suppression (NNS)](https://www.pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python/):

    * important becuase you will (with almost 100% certainty) detect multiple bounding boxes surrounding the object in the image.
    * a necessary layer for our rcnn
    * essentially we want to calculate the area of each squares and return only the one that overlaps the most
    ![NNS](images/nms_slow_01.jpg)


## Misc:
* vsc environment variable issues when env_vars are already used. Fix restart vscode

Leaky ReLU.

* Leaky ReLUs are one attempt to fix the “dying ReLU” problem. Instead of the function being zero when x < 0, a leaky ReLU will instead have a small negative slope (of 0.01, or so). so they learn faster. Once a ReLU ends up in this state, it is unlikely to recover, because the function gradient at 0 is also 0, so gradient descent learning will not alter the weights. "Leaky" ReLUs with a small positive gradient for negative inputs (y=0.01x when x < 0 say) are one attempt to address this issue and give a chance to recover.

(conv_102): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

* Kernel Size:
    * The kernel size here refers to the widthxheight of the filter mask.(so kernel size of 3 or 3,3 meansa 3x3)

* padding:
    * An alternative approach to applying a filter to an image is to ensure that each pixel in the image is given an opportunity to be at the center of the filter.
    * padding of 1 surrounds the image with 0 value which does not change the dot product but allows each cells to be the center once.
    * using a kernel size of 3 should be accompanied with padding of 1

* stride: (down-sampling)
    * the kernel movement Vertical and horizontal. 
    * the stride can be changed to (2,2). This has the effect of moving the filter two pixels right for each horizontal movement of the filter and two pixels down for each vertical movement of the filter 

* bias:
    * On a small network, of course you need a bias input, but on a large network, removing it makes almost no difference.

## how does it work?

Let us consider an example below, where the input image is 416 x 416, and stride of the network is 32. As pointed earlier, the dimensions of the feature map will be 13 x 13. We then divide the input image into 13 x 13 cells.

![yolo](images/yolo-5.png)

Then, the cell (on the input image) containing the center of the ground truth box of an object is chosen to be the one responsible for predicting the object. In the image, it is the cell which marked red, which contains the center of the ground truth box (marked yellow).

Now, the red cell is the 7th cell in the 7th row on the grid. We now assign the 7th cell in the 7th row on the feature map (corresponding cell on the feature map) as the one responsible for detecting the dog.

Now, this cell can predict three bounding boxes. Which one will be assigned to the dog's ground truth label?

## anchors
It might make sense to predict the width and the height of the bounding box, but in practice, that leads to unstable gradients during training. Instead, most of the modern object detectors predict log-space transforms, or simply offsets to pre-defined default bounding boxes called anchors.

Then, these transforms are applied to the anchor boxes to obtain the prediction. YOLO v3 has three anchors, which result in prediction of three bounding boxes per cell.

Making the predictions:
![formulas](images/yolo6.png)

* bx, by, bw, bh are the x,y center co-ordinates, width and height of our prediction. 
* tx, ty, tw, th is what the network outputs. cx and cy are the top-left co-ordinates of the grid. 
* pw and ph are anchors dimensions for the box.

## Center coordinates 
For example, consider the case of our dog image. If the prediction for center is (0.4, 0.7), then this means that the center lies at (6.4, 6.7) on the 13 x 13 feature map. (Since the top-left co-ordinates of the red cell are (6,6)).

But wait, what happens if the predicted x,y co-ordinates are greater than one, say (1.2, 0.7). This means center lies at (7.2, 6.7). Notice the center now lies in cell just right to our red cell, or the 8th cell in the 7th row. This breaks theory behind YOLO because if we postulate that the red box is responsible for predicting the dog, the center of the dog must lie in the red cell, and not in the one beside it.

Therefore, to remedy this problem, the output is passed through a sigmoid function, which squashes the output in a range from 0 to 1, effectively keeping the center in the grid which is predicting.

## Objectness Score

Object score represents the probability that an object is contained inside a bounding box. It should be nearly 1 for the red and the neighboring grids, whereas almost 0 for, say, the grid at the corners.

The objectness score is also passed through a sigmoid, as it is to be interpreted as a probability.

## Class Confidences (sigmoid(multiple) vs softmax(single))

Class confidences represent the probabilities of the detected object belonging to a particular class (Dog, cat, banana, car etc). Before v3, YOLO used to softmax the class scores.

However, that design choice has been dropped in v3, and authors have opted for using sigmoid instead. The reason is that Softmaxing class scores assume that the classes are mutually exclusive. In simple words, if an object belongs to one class, then it's guaranteed it cannot belong to another class. This is true for COCO database on which we will base our detector.

However, this assumptions may not hold when we have classes like Women and Person. This is the reason that authors have steered clear of using a Softmax activation.

## Prediction across different scales.

YOLO v3 makes prediction across 3 different scales. The detection layer is used make detection at feature maps of three different sizes, having strides 32, 16, 8 respectively. This means, with an input of 416 x 416, we make detections on scales 13 x 13, 26 x 26 and 52 x 52.
![](images/yolo_Scales-1.png)

## Output Processing

For an image of size 416 x 416, YOLO predicts ((52 x 52) + (26 x 26) + 13 x 13)) x 3 = 10647 bounding boxes. However, in case of our image, there's only one object, a dog. How do we reduce the detections from 10647 to 1?

## Thresholding by Object Confidence

First, we filter boxes based on their objectness score. Generally, boxes having scores below a threshold are ignored.

NMS intends to cure the problem of multiple detections of the same image. For example, all the 3 bounding boxes of the red grid cell may detect a box or the adjacent cells may detect the same object.

In [4]:
import torch
import os


In [5]:
import pandas

In [6]:
print('hello world')

hello world


In [7]:
torch.cuda.current_device()

0

In [8]:
torch.cuda.device(0)

In [10]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [11]:
torch.cuda.is_available()

True